<font size="30">---------------</font>
<font size="30">Data processing</font>
<font size="30">---------------</font>

In [1]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [ ]:
textFile = open('D:\SE_1502538_FYP2\Job Advert Data Collection\data_2019-03-01.txt', 'r+', encoding="utf-8")
text_data = textFile.read().lower()
textFile.close()

split_text_data = []

for word in text_data.split():
    split_text_data.append(word)
print(split_text_data[:5])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# our corpus
data = split_text_data

cv = CountVectorizer()

# convert text data into term-frequency matrix
data = cv.fit_transform(data)

tfidf_transformer = TfidfTransformer()

# convert term-frequency matrix into tf-idf
tfidf_matrix = tfidf_transformer.fit_transform(data)

# create dictionary to find a tfidf word each word
word2tfidf = dict(zip(cv.get_feature_names(), tfidf_transformer.idf_))

for word, score in word2tfidf.items():
    print(word, score)